# Прогнозирование рейтинга игрока

Проверка гипотезы, что на основании текущего рейтинга и данных об игроке определить его возможный рейтинг в следующем сезоне.
Данная информация поможет скауту/спортивному директору определиться с выбором при покупке игрока/продлении с ним контракта.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

### Анализ и очистка данных

***

In [2]:
df = pd.read_csv('data.csv', delimiter=',')
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                18207 non-null  int64  
 1   ID                        18207 non-null  int64  
 2   Name                      18207 non-null  object 
 3   Age                       18207 non-null  int64  
 4   Photo                     18207 non-null  object 
 5   Nationality               18207 non-null  object 
 6   Flag                      18207 non-null  object 
 7   Overall                   18207 non-null  int64  
 8   Potential                 18207 non-null  int64  
 9   Club                      17966 non-null  object 
 10  Club Logo                 18207 non-null  object 
 11  Value                     18207 non-null  object 
 12  Wage                      18207 non-null  object 
 13  Special                   18207 non-null  int64  
 14  Prefer

Удаляем заведомо не влияющие на футбольные качества игрока данные

In [4]:
drop_cols = df.columns[28:54]
df = df.drop(drop_cols, axis = 1)
df.drop(['Unnamed: 0',
         'Name',
         'Nationality',
         'Photo',
         'Flag',
         'Club Logo',
         'Club',
         'Real Face',
         'Value',
         'Wage',
         'Loaned From',
         'Joined',
         'Jersey Number',
         'Release Clause',
         'Contract Valid Until',
         'Body Type'],axis=1,inplace=True)
df.head()

,ID,Age,Overall,Potential,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,158023,31,94,94,2202,Left,5.0,4.0,4.0,Medium/ Medium,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,20801,33,94,94,2228,Right,5.0,4.0,5.0,High/ Low,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,190871,26,92,93,2143,Right,5.0,5.0,5.0,High/ Medium,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,193080,27,91,93,1471,Right,4.0,3.0,1.0,Medium/ Medium,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,192985,27,91,92,2281,Right,4.0,5.0,4.0,High/ High,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        18207 non-null  int64  
 1   Age                       18207 non-null  int64  
 2   Overall                   18207 non-null  int64  
 3   Potential                 18207 non-null  int64  
 4   Special                   18207 non-null  int64  
 5   Preferred Foot            18159 non-null  object 
 6   International Reputation  18159 non-null  float64
 7   Weak Foot                 18159 non-null  float64
 8   Skill Moves               18159 non-null  float64
 9   Work Rate                 18159 non-null  object 
 10  Position                  18147 non-null  object 
 11  Height                    18159 non-null  object 
 12  Weight                    18159 non-null  object 
 13  Crossing                  18159 non-null  float64
 14  Finish

#### Проверяем пустые значения

In [6]:
df[df['Height'].isnull()]

,ID,Age,Overall,Potential,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
13236,177971,33,62,62,1510,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13237,195380,29,62,62,1532,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13238,139317,35,62,62,1573,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13239,240437,20,62,72,1610,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13240,209462,24,62,66,1481,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13241,219702,21,62,72,1692,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13242,225590,23,62,70,1663,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13243,233782,19,62,78,1328,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13244,239158,30,62,62,1244,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13245,242998,21,62,69,1549,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Можно эти строки удалить

In [7]:
df.drop(df[df['Height'].isnull()].index.tolist(),inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18159 entries, 0 to 18206
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        18159 non-null  int64  
 1   Age                       18159 non-null  int64  
 2   Overall                   18159 non-null  int64  
 3   Potential                 18159 non-null  int64  
 4   Special                   18159 non-null  int64  
 5   Preferred Foot            18159 non-null  object 
 6   International Reputation  18159 non-null  float64
 7   Weak Foot                 18159 non-null  float64
 8   Skill Moves               18159 non-null  float64
 9   Work Rate                 18159 non-null  object 
 10  Position                  18147 non-null  object 
 11  Height                    18159 non-null  object 
 12  Weight                    18159 non-null  object 
 13  Crossing                  18159 non-null  float64
 14  Finish

В датасете осталось 12 строк с пустыми значениям в поле Position

In [8]:
df[df['Position'].isnull()]

,ID,Age,Overall,Potential,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
5018,153160,37,70,70,1574,Right,1.0,3.0,2.0,Medium/ Medium,...,77.0,64.0,79.0,70.0,70.0,16.0,8.0,11.0,12.0,13.0
6736,175393,33,68,68,1664,Left,1.0,3.0,3.0,High/ Low,...,69.0,67.0,12.0,34.0,33.0,16.0,13.0,15.0,16.0,8.0
7922,195905,33,67,67,1619,Right,1.0,3.0,2.0,Medium/ Medium,...,42.0,54.0,72.0,71.0,64.0,11.0,7.0,8.0,12.0,12.0
9905,226044,23,66,69,1632,Right,1.0,4.0,3.0,High/ Low,...,67.0,64.0,19.0,24.0,23.0,13.0,11.0,11.0,7.0,9.0
10628,216751,26,65,66,1536,Right,1.0,2.0,2.0,Medium/ Medium,...,39.0,57.0,68.0,69.0,68.0,14.0,12.0,11.0,14.0,12.0
16450,193911,31,57,57,1032,Right,1.0,3.0,1.0,Medium/ Medium,...,16.0,52.0,12.0,15.0,12.0,53.0,48.0,62.0,57.0,60.0
16539,245167,23,57,63,1410,Left,1.0,4.0,2.0,High/ Medium,...,38.0,57.0,60.0,61.0,57.0,15.0,12.0,11.0,6.0,12.0
16793,228192,31,56,56,1613,Right,1.0,3.0,2.0,Medium/ Medium,...,59.0,63.0,43.0,48.0,32.0,7.0,8.0,9.0,15.0,9.0
17129,228198,26,55,58,1466,Right,1.0,4.0,2.0,High/ Medium,...,65.0,42.0,26.0,18.0,13.0,15.0,14.0,8.0,10.0,16.0
17339,233526,23,54,63,1321,Right,1.0,3.0,2.0,Medium/ Low,...,58.0,45.0,14.0,23.0,21.0,8.0,6.0,13.0,9.0,10.0


Это меньше одной сотой процента от общего количества строк - можно удлалить не внеся больших изменений в датасет

In [13]:
df.dropna(inplace=True)
df

,ID,Age,Overall,Potential,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,158023,31,94,94,2202,Left,5.0,4.0,4.0,Medium/ Medium,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,20801,33,94,94,2228,Right,5.0,4.0,5.0,High/ Low,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,190871,26,92,93,2143,Right,5.0,5.0,5.0,High/ Medium,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,193080,27,91,93,1471,Right,4.0,3.0,1.0,Medium/ Medium,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,192985,27,91,92,2281,Right,4.0,5.0,4.0,High/ High,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,238813,19,47,65,1307,Right,1.0,2.0,2.0,Medium/ Medium,...,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0
18203,243165,19,47,63,1098,Right,1.0,2.0,2.0,Medium/ Medium,...,43.0,42.0,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0
18204,241638,16,47,67,1189,Right,1.0,3.0,2.0,Medium/ Medium,...,55.0,41.0,32.0,13.0,11.0,6.0,5.0,10.0,6.0,13.0
18205,246268,17,47,66,1228,Right,1.0,3.0,2.0,Medium/ Medium,...,50.0,46.0,20.0,25.0,27.0,14.0,6.0,14.0,8.0,9.0
